# Конкурсная работа №1: Кластеризация алгоритмом k-средних

#### Выполнил студент гр. 9381 Семенов Александр.

## Цель работы
#### Практическое применение алгоритма кластеризации k-средних на примере звуковых сигналов mp3-формата.

## Задачи работы

##### 1. С помощью ДПФ считать набор сигналов и превратить их в вектора.
##### 2. Прогнать на получившихся векторах алгоритм k-средних
##### 3. Записать результат: названия треков, разбитые по классам

## Данные для работы

 - Выборка сигналов mp3-формата (bitrate 256).
 - Формат записи — стерео-сигнал, 44100 гц.

 ## Выполнение работы
 
 ### 1) Подключение библиотек:

In [44]:
from pydub import AudioSegment
import scipy.fft
import numpy as np
import more_itertools as mit
from statistics import mean, StatisticsError
import random
from csv import writer
import copy

import pyprind

 ### 2) Считывание сигнала:

In [45]:
signals = []
signalNames = []

signalCount = 124;

bar = pyprind.ProgBar(signalCount, track_time = False, title = "Загрузка сигналов:")

for i in range(signalCount):
    audiofile = AudioSegment.from_mp3("dataSet/{:02}.mp3".format(i + 1)).set_channels(1)
    signals.append(np.array(audiofile.get_array_of_samples()))
    signalNames.append("{:02}.mp3".format(i + 1))
    bar.update()


Загрузка сигналов:
0% [##############################] 100%


 ### 3) Деление на окна и получение амплитудного спектра сигнала:

In [46]:
dimension = 256
windowOffset = 256
threshold = 0.2

amplitudeSpectrals = []
bar = pyprind.ProgBar(signalCount, track_time = False, title = "Обработка сигналов:")


for signal_k in range(signalCount):
    signalIntervals = list(mit.windowed(signals.pop(0), fillvalue = 0.0, n = dimension, step = windowOffset))
    
    bound = 0
    for interval in signalIntervals:
        m = np.max(interval)
        if m > bound:
            bound = m
    bound = bound * threshold
    signalIntervals = list(filter(lambda i: np.max(i) > bound, signalIntervals))
    
    signalIntervals = list(map(lambda interval: scipy.fft.fft(interval*np.hanning(len(interval))), signalIntervals))
    amplitudeSpectral = [[np.abs(i) for i in interval[:dimension//2]] for interval in signalIntervals]
    amplitudeSpectralMean = [0 for i in range(dimension//2)]
    
    for ASinterval in amplitudeSpectral:
        for i in range(dimension//2):
            amplitudeSpectralMean[i] += ASinterval[i]
    
    for i in range(dimension//2):
        amplitudeSpectralMean[i] = amplitudeSpectralMean[i] / len(signalIntervals)
    
    amplitudeSpectrals.append(amplitudeSpectralMean)
    bar.update()

Обработка сигналов:
0% [################              ] 100%

MemoryError: 

 ### 4) Описание класса-класстера:

In [40]:
class Claster:
    def __init__(self, centreVector, vectorSpace):
        self.centreVector = centreVector
        self.vectorSpace = vectorSpace
        self.vectorNames = set()
    
    def getCentreVec(self):
        return self.centreVector

    def getVecNames(self):
        return self.vectorNames

    def getVectors(self):
        return [self.vectorSpace[vectorName] for vectorName in self.vectorNames]
    
    def appendVec(self, vectorName):
        self.vectorNames.add(vectorName)
    
    def clearSetVecs(self):
        self.vectorNames.clear()

    def recalculateCentreVec(self):
        isChange = False
        for coord_i in range(len(self.centreVector)):
            try:
                newCoord = mean([self.vectorSpace[vectorName][coord_i] for vectorName in self.vectorNames])
            except StatisticsError:
                newCoord = self.centreVector[coord_i]
            
            if (self.centreVector[coord_i] != newCoord):
                isChange = True
            self.centreVector[coord_i] = newCoord
        
        return isChange

 ### 5) Подготовительные функции инициализации начальных центроидов и класстеров:

In [41]:
# Генерация начальных центроидов
def getCentreVectors(vectorSpace):
    vectors = list(vectorSpace.values())

    centreVecs = []
    for i in range(5):
        randInd = random.randrange(len(vectors)) # 5 рандомных векторов из множества становятся центроидами
        centreVecs.append(list(vectors.pop(randInd)))
    
    return centreVecs

# Генерация кластеров
def getStartClasters(centreVecs, vectorSpace):
    return [Claster(centreV, vectorSpace) for centreV in centreVecs]

 ### 6) Реализация алгоритма k-средних:

In [42]:
# Вычисление расстояния между векторами
def distance(V1, V2):
    ((V2[0] - V1[0])**2 + (V2[1] - V1[1])**2)**0.5
    return (sum([(V2[i] - V1[i])**2 for i in range(len(V1))]))**0.5

# Вычисление оценки кластеризации
def clusterCohesion(clasters):
    return mean([sum([distance(vector, claster.getCentreVec())**2 for vector in claster.getVectors()]) for claster in clasters])

# Разбиение на кластеры
def k_average(vectorSpace):
    clasters = getStartClasters(getCentreVectors(vectorSpace), vectorSpace)

    isContinue = True
    while(isContinue):
        for claster in clasters:
            claster.clearSetVecs()

        for vectorName in vectorSpace:
            distances = [distance(claster.getCentreVec(), vectorSpace[vectorName]) for claster in clasters]
            clasters[distances.index(min(distances))].appendVec(vectorName)
        
        isContinue = any([claster.recalculateCentreVec() for claster in clasters])
    
    return tuple([claster.getVecNames() for claster in clasters]), clusterCohesion(clasters)

 ### 7) Запуск алгоритма и запись результата в файл:

In [43]:
# Функция записи результата в файл
def writeToCSV(classes, estimate):
    with open("result.csv", "w", newline='') as file:
        writerObj = writer(file, delimiter = ';')
        for i in range(len(classes)):
            for vecName in classes[i]:
                writerObj.writerow([vecName, i+1])
                
        writerObj.writerow(["Оценка", estimate])


vectorSpace = dict()
iterCount = 200;
bar = pyprind.ProgBar(iterCount + 1, track_time = False, title = "Кластеризация треков:")

for i in range(signalCount):
    vectorSpace[signalNames.pop(0)] = tuple(amplitudeSpectrals.pop(0))

resClasses, resEstimate = k_average(vectorSpace)
bar.update()
for i in range(iterCount): # Выбираем кластеризацию с наименьшой оценкой
    classes, estimate = k_average(vectorSpace)
    if (estimate < resEstimate):
        resClasses, resEstimate = classes, estimate
    bar.update()

writeToCSV(resClasses, resEstimate)

Кластеризация треков:
0% [##############################] 100%


## Вывод
#### В ходе выполнения работы был изучен алгоритм кластеризации - k-средних; также была произведена кластеризация звуковых файлов формата mp3.